In [98]:
import pandas as pd
import nltk.data
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout, Flatten
from keras.layers import Activation
import numpy as np
import chardet
import logging
from keras.models import Model
from keras.layers import Input,Dense,Dropout,Flatten
from keras.layers import Conv1D,MaxPooling1D,AveragePooling1D
from keras.layers import LSTM,Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
import datetime
import time
import os

In [2]:
def read_data(filename):
    df = pd.read_csv(filename)
    return df

In [3]:
def stemming(text):    
    text = [stemmer.stem(word) for word in text]
    return " ".join(text)

In [4]:
def cleantext(dataframe,stem):
#Cleaning text    
    dataframe["clean_text"] = dataframe["docket_text"]
#Get bundle number
    dataframe['bundle_motion_number'] = dataframe['bundle_motion_number'].apply(lambda x: np.nan if x == 'None' else float(x))
#Replace bundle motion number
    dataframe.clean_text = dataframe.apply(lambda x: x.clean_text if np.isnan(x.bundle_motion_number) 
                                           else x.clean_text.replace(str(int(x.bundle_motion_number)), 'DLLM'), axis = 1)
#Remove numbers
    dataframe.clean_text = dataframe.clean_text.str.replace(r'\d+','')
#Remove floats
    dataframe.clean_text = dataframe.clean_text.astype(str).replace(r'(\d*\.?\d*)','')
#Remove Punctuation
    dataframe.clean_text = dataframe.clean_text.astype(str).apply(lambda x : " ".join(re.findall('[\w]+',x)))
#Convert to lower text
    dataframe.clean_text = dataframe.clean_text.astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))      
#Stop words removal
    dataframe.clean_text.astype(str)
    stop = stopwords.words('english')
#     dataframe.loc[:,"docket_text_new"] = dataframe.loc[:,"docket_text_new"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    dataframe.clean_text = dataframe.clean_text.apply(lambda x: " ".join(x.lower() for x in str(x).split() if x not in stop))        
# Adding "Start" & "End" at the beginning & end of each docket text
    dataframe.clean_text = " START " + dataframe.clean_text + " END"
    dataframe.clean_text = [nltk.word_tokenize(i) for i in dataframe.clean_text]
# Moving cleaned text to another column
#     dataframe['cleaned_text'] = dataframe["docket_text_new"]
    if stem == True:
        dataframe.clean_text = dataframe.clean_text.apply(stemming)
        dataframe.clean_text.head(5)
        ##Stemming
        stemmer = SnowballStemmer("english")  


In [5]:
def clean_text(dataframe,stem,stop,text_col):
#Cleaning text    
    dataframe["doc"] = dataframe[text_col]
#Remove numbers
    dataframe.loc[:,"doc"] = dataframe.doc.str.replace(r'\d+','')
#Remove floats
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).replace(r'(\d*\.?\d*)','')
#Remove Punctuation
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x : " ".join(re.findall('[\w]+',x)))
#Convert to lower text
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))      
#Stop words removal
    if stop == True:
        dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].astype(str)
        stop = stopwords.words('english')
        dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].apply(lambda x: " ".join(x.lower() for x in str(x).split() if x not in stop))
#Stemming        
    if stem == True:
        stemmer = SnowballStemmer("english")
        curried_stemming = curry(stemming, stemmer=stemmer)
        dataframe.loc[:,"doc"] = dataframe.loc[:,"docket_text_new"].apply(curried_stemming) 
# Adding "Start" & "End" at the beginning & end of each docket text
    dataframe.loc[:,"doc"] = " START " + dataframe.loc[:,"doc"] + " END"
# Moving cleaned text to another column
    dataframe['cleaned_text'] = dataframe["doc"]
#Tokenization                               
    dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].apply(nltk.word_tokenize)
    return dataframe

In [6]:
def tokenizetext(dataframe):    
    dataframe.clean_text = [nltk.word_tokenize(i) for i in dataframe.clean_text]

In [7]:
def get_list(dataframe):
    words =  dataframe["clean_text"].tolist()
    flat_list_words = [item for sublist in words for item in sublist]
    return flat_list_words

In [8]:
def w2c_model(doc,size,window,min_count,workers,iter,sg,length,numepoch):    
    model = gensim.models.Word2Vec(size=size, window=window, min_count=min_count, workers=workers,iter=iter,sg=sg)
    model.build_vocab(doc)
    model.train(doc,total_examples=length,epochs = numepoch)
    return model

In [9]:
def text2seq(doc, w2v, seq_len, emb_size):
    padding = np.array([0 for __ in range(emb_size)])
    seq = []
    for token in doc[: seq_len]:
        try:
            seq.append(w2v.wv[token])
        except:
            aa = 1

    seq += [padding for __ in range(seq_len - len(seq))]
    #print(len(seq))
    return seq

In [10]:
def one_hot(series):
    values=sorted(list(set(series)))
    return np.vstack(map(lambda x: [x==v for v in values],series))

In [11]:
def prepro(dataframe, w2v,seq_len, emb_size,target_col):
    X = dataframe['clean_text'].apply(lambda doc: text2seq(doc, w2v, seq_len, emb_size))
    X = np.vstack(X).reshape(len(dataframe), seq_len, emb_size)
    y = one_hot(dataframe[target_col])
    return X, y

In [12]:
#Model building 
def model1(seq_len, emb_size, num_labels):
    input_layer = Input(shape=(seq_len, emb_size))
    x = Conv1D(32, 3, activation='relu')(input_layer)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_labels, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [13]:
def build_model2(seq_len, emb_size, num_labels):
    input_layer = Input(shape=(seq_len, emb_size))
    x = Conv1D(32, 3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)  
    x = LSTM(100, return_sequences = True,dropout=0.2,recurrent_dropout=0.05)(input_layer)
    x = Flatten()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.2)(x)
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.2)(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)
    output_layer = Dense(num_labels, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [14]:
def build_model(seq_len, emb_size, num_labels):
    input_layer = Input(shape=(seq_len, emb_size))
    x = Conv1D(32, 3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)  
    x = LSTM(100, dropout=0.2, recurrent_dropout=0.2)(x) 
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_labels, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [15]:
def fit_model(X,y,weightsfile,num_labels,seq_len,emb_size,epochs):
    checkpoint = ModelCheckpoint(weightsfile, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model_dl = build_model2(seq_len, emb_size, num_labels)
    model_dl.fit(X, y, epochs=epochs, batch_size=16,callbacks=callbacks_list, validation_split=0.2)
    return model_dl

In [16]:
def get_type(dataframe):
    dataframe['type'] = dataframe['type'].apply(lambda x: x if x in ['Motion', 'RR', 'Order'] else 'None')  
#     dataframe['clean_text'] = dataframe["docket_text"]

In [17]:
def retain_summary_order(dataframe):
    Search_for_These_values = ['summary', 'order'] #creating list
    pattern = '|'.join(Search_for_These_values)    # joining list for comparision
    dataframe['motionOrsummary'] = dataframe["clean_text"].str.contains(pattern)
    dataframe['motionOrsummary'] = dataframe['motionOrsummary'].map({True: 'Yes', False: 'No'})
    dataframe = dataframe[dataframe['motionOrsummary']== 'Yes']
    return dataframe 

In [18]:
# def predict(model,weightsfile,csvfile,genismfile):
#     model.load_weights(weightsfile)
#     df = read_data(csvfile)
#     get_type(df)
#     cleantext(df,False)
#     df = retain_summary_order(df)
#     tokenizetext(df)
#     vocab = get_list(df)
#     w2c_model = build_genism_model(vocab = vocab,size=100,window=5,min_count=1,workers=6,iter=10,sg=0,length=length,numepoch=10)
#     X1,y1 = prepro(df,gen_model,seq_len=100, emb_size=100)
#     preds = model.predict(X1)
#     return preds,df

In [19]:
def prediction(model,weightsfile,X):
    model.load_weights(weightsfile)
    preds = model.predict(X)
    lst = list(map(lambda x: x.argmax(), preds))
    return preds,lst

In [20]:
def get_prediction_type(preds,dataframe):
    lst = list(map(lambda x: x.argmax(), preds))
#     dataframe['prediction']= lst
    values=sorted(list(set(dataframe['type'])))
    return values

In [21]:
def assign_label(dataframe):
    dataframe['prediction']= lst
    p_class = {0:'Motion',1:'None',2:'Order',3:'RR'}
    dataframe = dataframe.replace({'prediction': p_class}) 
    return dataframe

# Dealing with DATA

In [22]:
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 1000

In [372]:
data = read_data('docket_data8.csv')

In [46]:
# db = read_data('JDreview.csv')

In [47]:
# db.rename(columns={'type': 'bundle_type'}, inplace=True)

In [374]:
# db = db.drop(['clean_text','motionOrsummary','bundle_action_prediction','bundle_decision_prediction','Output Review','Corrected bundle_action_prediction','Corrected bundle_decision_prediction','Notes','Column3'],axis=1)
data.groupby('nature_of_suit').count()


,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,bundle_action,bundle_type,bundle_who_filed,mtd_action,mtd_type
nature_of_suit,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310,839,2310,2310,2310,2310,2310,2310,2310,2310,2310,2310
1,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35
110 Insurance,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,36,65,65,65,65,65,65,65,65,65,65
111 Miscellaneous,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,0,3,3,3,3,3
190 Contract: Other,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73
2,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,0,68,68,68,68,68,68,68,68,68,68
320 Assault Libel & Slander,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
360 P.I.: Other,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,0,12,12,12,12,12,12,12,12,12,12
365 Personal Inj. Prod. Liability,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [376]:
data.groupby('bundle_action').count()

,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_type,bundle_who_filed,mtd_action,mtd_type
bundle_action,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,3037,2986,3037,1459,3037,3037,3037,3037,3034,3037,3037,3037,3037,3037
Grant,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2881,2844,2881,1395,2881,2881,2881,2881,2877,2881,2881,2881,2881,2881
None,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2236134,2233913,2236134,2236134,2208326,2236134,1098294,2236134,2236134,2233919,2236122,2229100,2233857,2236134,2236134,2236134,2236134
Partial,937,937,937,937,937,937,937,937,937,937,937,937,937,937,937,937,922,937,436,937,937,937,937,934,937,937,937,937,937


In [375]:
k_class = {'granted':'Grant','denied':'Deny','partial':'Partial'}
data = data.replace({'bundle_action': k_class})

In [378]:
# data = da.append(db, ignore_index=True)
data = data.drop_duplicates(['case_number', 'activity_number','bundle_motion_number','docket_text'], keep='last')

In [380]:
data.to_csv("week15_data.csv")

In [618]:
len(data)

2237527

In [617]:
len(data)

2237527

# Word2Vec

In [23]:
start_time = time.time()

data = read_data('week15_data.csv')

cleantext(data,stem = False)

# tokenizetext(df)

# doc = get_list(df)

doc = data.clean_text

w2c = w2c_model(doc,size=100,window=5,min_count=1,workers=6,iter=10,sg=0,length=len(doc),numepoch=10)

runtime = time.time() - start_time
a = runtime/60

print("Done with w2c model!")
print("It took",a, "mins" )

Done with w2c model!
It took 30.141986870765685 mins


In [25]:
# runtime = time.time() - start_time
# runtime/60

In [89]:
# print (da.clean_text.apply(lambda x: 'dllm' in x.lower()).sum())

6656


In [342]:
data = read_data('week13_data.csv')

In [341]:
# #Cleaning text    
# da["clean_text"] = da["docket_text"]
# #Get bundle number
# da['bundle_motion_number'] = da['bundle_motion_number'].apply(lambda x: np.nan if x == 'None' else float(x))
# #Replace bundle motion number
# da.clean_text = da.apply(lambda x: x.clean_text if np.isnan(x.bundle_motion_number) 
#                         else x.clean_text.replace(str(int(x.bundle_motion_number)), 'DLLM'), axis = 1)

In [34]:
# for i in range (int((len(data)/ batch_size))):
#     data_batch = data[data['case_number'].isin(cases[i * batch_size: (i + 1) * batch_size])].copy()

# Train Model (Bundle Type)

In [57]:
# ds = read_data('train.csv')

# d1 = ds[ds['type'] != 'None']
# d2 = ds[ds['type'] == 'None'].sample(6000)
# ds = d1.append(d2, ignore_index=True)
# ds = ds.sample(frac=1)

# cleantext(ds,stem=False)

# X, y = prepro(ds,w2c,seq_len=100, emb_size=100,target_col='type')


# CNN_model= fit_model(X=X,y=y,weightsfile = "weights.best2.hdf5",num_labels=4,seq_len=100,emb_size=100,epochs=20)

# print("Done with training!")


In [102]:
# df = read_data('week12_data.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (3,4,5,6,10,12,20,21,23,24,25,26,27,30,31,32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [382]:
df = data[data['docket_text'].apply(lambda x: any(y in str(x).lower() for y in ['summary', 'order','dismiss']))]
# cleantext(df,stem = False)

In [383]:
df.groupby('bundle_type').count()

,Unnamed: 0,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_who_filed,mtd_action,mtd_type,clean_text
bundle_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,10158,10158,10158,10158,10071,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,9988,10158,4991,10158,10158,10158,10158,10134,10158,10158,10158,10158,10158,10158
None,837932,837932,837932,837932,8,837932,837932,837932,837932,837932,837932,837932,837932,837932,837932,837932,837932,826837,837932,441874,837932,837932,837932,837930,835396,837222,837932,837932,837932,837932,837932
Order,7696,7696,7696,7696,5926,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7564,7696,3762,7696,7696,7696,7696,7679,7696,7696,7696,7696,7696,7696
RR,240,240,240,240,130,240,240,240,240,240,240,240,240,240,240,240,240,238,240,150,240,240,240,240,240,240,240,240,240,240,240


In [384]:
len(df)

856026

In [385]:
d1 = df[df['bundle_type'] != 'None']
d2 = df[df['bundle_type'] == 'None'].sample(12000)
df = d1.append(d2, ignore_index=True)
df = df.sample(frac=1)

In [386]:
df.groupby('bundle_type').count()

,Unnamed: 0,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_who_filed,mtd_action,mtd_type,clean_text
bundle_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,10158,10158,10158,10158,10071,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,9988,10158,4991,10158,10158,10158,10158,10134,10158,10158,10158,10158,10158,10158
None,12000,12000,12000,12000,0,12000,12000,12000,12000,12000,12000,12000,12000,12000,12000,12000,12000,11845,12000,6366,12000,12000,12000,12000,11966,11990,12000,12000,12000,12000,12000
Order,7696,7696,7696,7696,5926,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7564,7696,3762,7696,7696,7696,7696,7679,7696,7696,7696,7696,7696,7696
RR,240,240,240,240,130,240,240,240,240,240,240,240,240,240,240,240,240,238,240,150,240,240,240,240,240,240,240,240,240,240,240


In [66]:
# k_class = {'Transfer-Remand':'TransferRemand','Admin Closing':'AdminClosing','SJ Submission':'SJSubmission'}
# df = df.replace({'bundle_type': k_class})

In [387]:
X, y = prepro(df,w2c,seq_len=100, emb_size=100,target_col='bundle_type')

In [388]:
start_time = time.time()

CNN_model= fit_model(X=X,y=y,weightsfile = "bundle_type.hdf5",num_labels=4,seq_len=100,emb_size=100,epochs=20)

runtime = time.time() - start_time
a = runtime/60

print("Done with training!")
print("It took",a, "mins" )

Train on 24075 samples, validate on 6019 samples
Epoch 1/20
24075/24075 [==============================] - 173s 7ms/step - loss: 0.0911 - acc: 0.9710 - val_loss: 0.0621 - val_acc: 0.9864

Epoch 00001: val_acc improved from -inf to 0.98638, saving model to bundle_type.hdf5
Epoch 2/20
24075/24075 [==============================] - 151s 6ms/step - loss: 0.0323 - acc: 0.9898 - val_loss: 0.0399 - val_acc: 0.9880

Epoch 00002: val_acc improved from 0.98638 to 0.98804, saving model to bundle_type.hdf5
Epoch 3/20
24075/24075 [==============================] - 151s 6ms/step - loss: 0.0220 - acc: 0.9929 - val_loss: 0.0450 - val_acc: 0.9889

Epoch 00003: val_acc improved from 0.98804 to 0.98887, saving model to bundle_type.hdf5
Epoch 4/20
24075/24075 [==============================] - 150s 6ms/step - loss: 0.0164 - acc: 0.9951 - val_loss: 0.0536 - val_acc: 0.9880

Epoch 00004: val_acc did not improve from 0.98887
Epoch 5/20
24075/24075 [==============================] - 151s 6ms/step - loss: 0.01

# Training Model (MTD Type)

In [24]:
dt = read_data('prediction.csv')


In [25]:
len(dt)

855930

In [29]:
dt.groupby('mtd_type').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction
mtd_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,14041,14041,14041,14041,14041,14041,14041,14041,14041,4,14041,14041,14041,14041,14041,14041,14041,14041,14041,14041,14041,14041,13895,14041,6807,14041,14041,14041,14041,13996,14039,14041,14041,14041,14041,14041,14041,14041,3,131
None,14000,14000,14000,14000,14000,14000,14000,14000,14000,257,14000,14000,14000,14000,14000,14000,14000,14000,14000,14000,14000,14000,13820,14000,7362,14000,14000,14000,14000,13964,13987,14000,14000,14000,14000,14000,14000,14000,186,385
Order,17622,17622,17622,17622,17622,17622,17622,17622,17622,690,17622,17622,17622,17622,17622,17622,17622,17622,17622,17622,17622,17622,17432,17622,8360,17622,17622,17622,17622,17585,17618,17622,17622,17622,17622,17622,17622,17622,1209,1258
RR,9,9,9,9,9,9,9,9,9,0,9,9,9,9,9,9,9,9,9,9,9,9,9,9,7,9,9,9,9,9,9,9,9,9,9,9,9,9,0,1


In [27]:
d1 = dt[dt['mtd_type'] != 'None']
d2 = dt[dt['mtd_type'] == 'None'].sample(14000)
dt = d1.append(d2, ignore_index=True)
dt = dt.sample(frac=1)

In [28]:
len(dt)

45672

In [30]:
X, y = prepro(dt,w2c,seq_len=100, emb_size=100,target_col='mtd_type')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
start_time = time.time()

CNN_model= fit_model(X=X,y=y,weightsfile = "mtd_type.hdf5",num_labels=4,seq_len=100,emb_size=100,epochs=20)

runtime = time.time() - start_time
a = runtime/60

print("Done with training!")
print("It took",a, "mins" )

Train on 36537 samples, validate on 9135 samples
Epoch 1/20
36537/36537 [==============================] - 337s 9ms/step - loss: 0.3468 - acc: 0.8584 - val_loss: 0.2896 - val_acc: 0.8852

Epoch 00001: val_acc improved from -inf to 0.88517, saving model to mtd_type.hdf5
Epoch 2/20
36537/36537 [==============================] - 309s 8ms/step - loss: 0.2685 - acc: 0.8954 - val_loss: 0.2680 - val_acc: 0.8956

Epoch 00002: val_acc improved from 0.88517 to 0.89557, saving model to mtd_type.hdf5
Epoch 3/20
36537/36537 [==============================] - 270s 7ms/step - loss: 0.2439 - acc: 0.9035 - val_loss: 0.2654 - val_acc: 0.8983

Epoch 00003: val_acc improved from 0.89557 to 0.89830, saving model to mtd_type.hdf5
Epoch 4/20
36537/36537 [==============================] - 328s 9ms/step - loss: 0.2267 - acc: 0.9110 - val_loss: 0.2615 - val_acc: 0.9005

Epoch 00004: val_acc improved from 0.89830 to 0.90049, saving model to mtd_type.hdf5
Epoch 5/20
36537/36537 [==============================] - 

# Training Model (Bundle_action)

In [79]:
df = read_data('prediction.csv')


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [80]:
len(ds)

15420

In [81]:
# data['docket_text'] = data.apply(lambda x: x.docket_text.replace(str(x.bundle_motion_number), ' dllm '), axis=1)

In [82]:
ds = df[(df['bundle_type']=='Order')]


In [83]:

d1 = ds[ds['bundle_action'] != 'None']
d2 = ds[ds['bundle_action'] == 'None'].sample(1900)
ds = d1.append(d2, ignore_index=True)


In [84]:
ds = ds.sample(frac=1)

In [85]:
ds.groupby('bundle_action').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction,mtd_action_prediction
bundle_action,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2248,2208,2248,1094,2248,2248,2248,2248,2245,2248,2248,2248,2248,2248,2248,2248,2248,2246,2246,299
Grant,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2176,2206,1069,2206,2206,2206,2206,2202,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,219
None,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900,530,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900,1862,1900,953,1900,1900,1900,1900,1893,1900,1900,1900,1900,1900,1900,1900,1900,1882,1882,206
Partial,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,757,768,371,768,768,768,768,765,768,768,768,768,768,768,768,768,768,768,116


In [86]:
X, y = prepro(ds,w2c,seq_len=100, emb_size=100,target_col='bundle_action')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
CNN_model= fit_model(X=X,y=y,weightsfile = "bundle_action.hdf5",num_labels=4,seq_len=100,emb_size=100,epochs=20)

Train on 5697 samples, validate on 1425 samples
Epoch 1/20
5697/5697 [==============================] - 59s 10ms/step - loss: 0.8653 - acc: 0.6349 - val_loss: 0.7694 - val_acc: 0.7095

Epoch 00001: val_acc improved from -inf to 0.70947, saving model to bundle_action.hdf5
Epoch 2/20
5697/5697 [==============================] - 55s 10ms/step - loss: 0.6802 - acc: 0.7267 - val_loss: 0.7041 - val_acc: 0.7207

Epoch 00002: val_acc improved from 0.70947 to 0.72070, saving model to bundle_action.hdf5
Epoch 3/20
5697/5697 [==============================] - 42s 7ms/step - loss: 0.6078 - acc: 0.7581 - val_loss: 0.6890 - val_acc: 0.7270

Epoch 00003: val_acc improved from 0.72070 to 0.72702, saving model to bundle_action.hdf5
Epoch 4/20
5697/5697 [==============================] - 41s 7ms/step - loss: 0.5549 - acc: 0.7804 - val_loss: 0.6703 - val_acc: 0.7467

Epoch 00004: val_acc improved from 0.72702 to 0.74667, saving model to bundle_action.hdf5
Epoch 5/20
5697/5697 [===========================

# Training Model (MTD Action)

In [44]:
df = read_data('prediction.csv')


In [45]:
ds = df[(df['mtd_type']=='Order')]


In [46]:
len(ds)

17622

In [47]:
ds.groupby('mtd_action').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction
mtd_action,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,3959,3959,3959,3959,3959,3959,3959,3959,3959,159,3959,3959,3959,3959,3959,3959,3959,3959,3959,3959,3959,3959,3915,3959,1885,3959,3959,3959,3959,3951,3959,3959,3959,3959,3959,3959,3959,3959,315,331
Grant,9102,9102,9102,9102,9102,9102,9102,9102,9102,258,9102,9102,9102,9102,9102,9102,9102,9102,9102,9102,9102,9102,9016,9102,4011,9102,9102,9102,9102,9084,9100,9102,9102,9102,9102,9102,9102,9102,484,503
Moot,2250,2250,2250,2250,2250,2250,2250,2250,2250,100,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250,2224,2250,1131,2250,2250,2250,2250,2247,2250,2250,2250,2250,2250,2250,2250,2250,153,159
None,412,412,412,412,412,412,412,412,412,6,412,412,412,412,412,412,412,412,412,412,412,412,410,412,228,412,412,412,412,410,410,412,412,412,412,412,412,412,16,16
Partial,1899,1899,1899,1899,1899,1899,1899,1899,1899,167,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,1867,1899,1105,1899,1899,1899,1899,1893,1899,1899,1899,1899,1899,1899,1899,1899,241,249


In [48]:
d1 = ds[ds['mtd_action'] != 'Grant']
d2 = ds[ds['mtd_action'] == 'Grant'].sample(6900)
ds = d1.append(d2, ignore_index=True)
ds = ds.sample(frac=1)

In [49]:
X, y = prepro(ds,w2c,seq_len=100, emb_size=100,target_col='mtd_action')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
CNN_model= fit_model(X=X,y=y,weightsfile = "mtd_action.hdf5",num_labels=5,seq_len=100,emb_size=100,epochs=20)

Train on 12336 samples, validate on 3084 samples
Epoch 1/20
12336/12336 [==============================] - 102s 8ms/step - loss: 0.8988 - acc: 0.6669 - val_loss: 0.7757 - val_acc: 0.7228

Epoch 00001: val_acc improved from -inf to 0.72276, saving model to mtd_action.hdf5
Epoch 2/20
12336/12336 [==============================] - 122s 10ms/step - loss: 0.7541 - acc: 0.7297 - val_loss: 0.7574 - val_acc: 0.7267

Epoch 00002: val_acc improved from 0.72276 to 0.72665, saving model to mtd_action.hdf5
Epoch 3/20
12336/12336 [==============================] - 92s 7ms/step - loss: 0.6997 - acc: 0.7430 - val_loss: 0.7552 - val_acc: 0.7305

Epoch 00003: val_acc improved from 0.72665 to 0.73054, saving model to mtd_action.hdf5
Epoch 4/20
12336/12336 [==============================] - 116s 9ms/step - loss: 0.6603 - acc: 0.7555 - val_loss: 0.7751 - val_acc: 0.7257

Epoch 00004: val_acc did not improve from 0.73054
Epoch 5/20
12336/12336 [==============================] - 101s 8ms/step - loss: 0.6266 

# Training Model (Filing_Party)

In [537]:
df = read_data('prediction.csv')


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [538]:
ds = df[df['bundle_type'] != 'None']

In [559]:
ds.groupby('bundle_who_filed').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction
bundle_who_filed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
None,3000,3000,3000,3000,3000,3000,3000,2476,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,2946,3000,1541,3000,3000,3000,3000,2992,3000,3000,3000,3000,3000,3000,3000,3000,1628
X,1595,1595,1595,1595,1595,1595,1595,1401,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1563,1595,697,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1514
defendant,6176,6176,6176,6176,6176,6176,6176,5764,6176,6176,6176,6176,6176,6176,6176,6176,6176,6176,6176,6176,6075,6176,3051,6176,6176,6176,6176,6163,6176,6176,6176,6176,6176,6176,6176,6176,1691
plaintiff,3682,3682,3682,3682,3682,3682,3682,3457,3682,3682,3682,3682,3682,3682,3682,3682,3682,3682,3682,3682,3625,3682,1739,3682,3682,3682,3682,3676,3682,3682,3682,3682,3682,3682,3682,3682,877


In [240]:
len(ds)

14245

In [540]:
d1 = ds[ds['bundle_who_filed'] != 'None']
d2 = ds[ds['bundle_who_filed'] == 'None'].sample(3000)
ds = d1.append(d2, ignore_index=True)
ds = ds.sample(frac=1)

In [541]:
X, y = prepro(ds,w2c,seq_len=100, emb_size=100,target_col='bundle_who_filed')

In [542]:
CNN_model= fit_model(X=X,y=y,weightsfile = "bundle_filed.hdf5",num_labels=4,seq_len=100,emb_size=100,epochs=20)

Train on 11562 samples, validate on 2891 samples
Epoch 1/20
11562/11562 [==============================] - 83s 7ms/step - loss: 0.7727 - acc: 0.7017 - val_loss: 0.7160 - val_acc: 0.7312

Epoch 00001: val_acc improved from -inf to 0.73123, saving model to bundle_filed.hdf5
Epoch 2/20
11562/11562 [==============================] - 72s 6ms/step - loss: 0.6634 - acc: 0.7492 - val_loss: 0.6762 - val_acc: 0.7430

Epoch 00002: val_acc improved from 0.73123 to 0.74300, saving model to bundle_filed.hdf5
Epoch 3/20
11562/11562 [==============================] - 73s 6ms/step - loss: 0.6158 - acc: 0.7663 - val_loss: 0.6732 - val_acc: 0.7465

Epoch 00003: val_acc improved from 0.74300 to 0.74645, saving model to bundle_filed.hdf5
Epoch 4/20
11562/11562 [==============================] - 73s 6ms/step - loss: 0.5850 - acc: 0.7785 - val_loss: 0.6572 - val_acc: 0.7548

Epoch 00004: val_acc improved from 0.74645 to 0.75476, saving model to bundle_filed.hdf5
Epoch 5/20
11562/11562 [======================

# Prediction

In [53]:
# db = read_data('bundle_pt1.csv')

# get_type(db)

# db = db[db['docket_text'].apply(lambda x: any(y in str(x).lower() for y in ['summary', 'order']))]

# cleantext(db,stem=False)

# X1, y1 = prepro(db,w2c,seq_len=100, emb_size=100,target_col='type')

# preds,lst = prediction(model=CNN_model,weightsfile = "weights.best2.hdf5",X = X1)

# db = assign_label(db)

# runtime = time.time() - start_time

# print('It took',(runtime/60),'minutes to run!')

# print('Done with prediction!')

In [389]:
# db = read_data('week12_data.csv')

da = data[data['docket_text'].apply(lambda x: any(y in str(x).lower() for y in ['summary', 'order','dismiss']))]

# cleantext(da,stem = False)


In [390]:
len(da)

856026

In [56]:
# da.sample(frac = 0.5, random_state = 0)

In [61]:
da = da.reset_index(drop=True)

In [391]:
def batch_predict(data, batch_size):
# get unique cases
    cases = data.case_number.unique()

# Create new data file
    new_data = pd.DataFrame()

    for i in range (int(len(cases) / batch_size)):
# slide through the cases list with a slide size of batch_size
        batch_cases = cases[i * batch_size: (i + 1) * batch_size]

#create a temp data frame of the data with just the cases
        d1 = data[data['case_number'].isin(batch_cases)].copy()
    
        X1, y1 = prepro(d1,w2c,seq_len=100, emb_size=100,target_col='bundle_type')
# Modify this next line to make it work with your code
        pred, d1['predicted_label'] = prediction(model=CNN_model,weightsfile = "bundle_type.hdf5",X = X1)

# Append predicted data to the new dataframe
        new_data = new_data.append(d1, ignore_index=True)

    return new_data

In [392]:
# X1, y1 = prepro(da,w2c,seq_len=100, emb_size=100,target_col='bundle_type')

new_data = batch_predict(da, batch_size= 50)
# preds, lst = prediction(model=CNN_model,weightsfile = "weights.best2.hdf5",X = X1)

In [393]:
len(new_data)

855930

In [395]:
len(da)

856026

In [48]:
# d1 = db[db['bundle_type'] != 'None']
# d2 = db[db['bundle_type'] == 'None'].sample(60000)
# db = d1.append(d2, ignore_index=True)
# db = db.sample(frac=1)

In [49]:
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [32]:
# da['type_prediction']= lst

In [396]:
values=sorted(list(set(new_data['bundle_type'])))
values

['Motion', 'None', 'Order', 'RR']

In [397]:
p_class = {0:'Motion',1:'None',2:'Order',3:'RR'}
new_data = new_data.replace({'predicted_label': p_class})


In [398]:
new_data.groupby('predicted_label').count()

,Unnamed: 0,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text
predicted_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,11473,11473,11473,11473,10066,11473,11473,11473,11473,11473,11473,11473,11473,11473,11473,11473,11473,11275,11473,5596,11473,11473,11473,11473,11444,11473,11473,11473,11473,11473,11473,11473
None,830698,830698,830698,830698,14,830698,830698,830698,830698,830698,830698,830698,830698,830698,830698,830698,830698,819755,830698,438295,830698,830698,830698,830696,828186,829994,830698,830698,830698,830698,830698,830698
Order,13000,13000,13000,13000,5927,13000,13000,13000,13000,13000,13000,13000,13000,13000,13000,13000,13000,12755,13000,6389,13000,13000,13000,13000,12964,12994,13000,13000,13000,13000,13000,13000
RR,759,759,759,759,128,759,759,759,759,759,759,759,759,759,759,759,759,746,759,497,759,759,759,759,759,759,759,759,759,759,759,759


In [399]:
new_data.groupby('bundle_type').count()

,Unnamed: 0,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label
bundle_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,10158,10158,10158,10158,10071,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,10158,9988,10158,4991,10158,10158,10158,10158,10134,10158,10158,10158,10158,10158,10158,10158
None,837836,837836,837836,837836,8,837836,837836,837836,837836,837836,837836,837836,837836,837836,837836,837836,837836,826741,837836,441874,837836,837836,837836,837834,835300,837126,837836,837836,837836,837836,837836,837836
Order,7696,7696,7696,7696,5926,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7696,7564,7696,3762,7696,7696,7696,7696,7679,7696,7696,7696,7696,7696,7696,7696
RR,240,240,240,240,130,240,240,240,240,240,240,240,240,240,240,240,240,238,240,150,240,240,240,240,240,240,240,240,240,240,240,240


In [402]:
new_data.to_csv("prediction.csv")

# MTD Prediction

In [32]:
db = read_data('prediction.csv')


In [33]:
def batch_predict_mtd(data, batch_size):
# get unique cases
    cases = data.case_number.unique()

# Create new data file
    new_data = pd.DataFrame()

    for i in range (int(len(cases) / batch_size)):
# slide through the cases list with a slide size of batch_size
        batch_cases = cases[i * batch_size: (i + 1) * batch_size]

#create a temp data frame of the data with just the cases
        d1 = data[data['case_number'].isin(batch_cases)].copy()
    
        X1, y1 = prepro(d1,w2c,seq_len=100, emb_size=100,target_col='mtd_type')
# Modify this next line to make it work with your code
        pred, d1['mtd_predicted'] = prediction(model=CNN_model,weightsfile = "mtd_type.hdf5",X = X1)

# Append predicted data to the new dataframe
        new_data = new_data.append(d1, ignore_index=True)

    return new_data

In [34]:
len(db)

855930

In [35]:
mtd_data = batch_predict_mtd(db, batch_size= 50)
print('done')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


done


In [36]:
len(mtd_data)

855930

In [37]:
values=sorted(list(set(mtd_data['mtd_type'])))
values

['Motion', 'None', 'Order', 'RR']

In [38]:
p_class = {0:'Motion',1:'None',2:'Order', 3:'RR'}
mtd_data = mtd_data.replace({'mtd_predicted': p_class})

In [51]:
mtd_data.to_csv("prediction.csv")

In [39]:
mtd_data.groupby('mtd_type').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction
mtd_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,14041,14041,14041,14041,14041,14041,14041,14041,14041,4,14041,14041,14041,14041,14041,14041,14041,14041,14041,14041,14041,14041,13895,14041,6807,14041,14041,14041,14041,13996,14039,14041,14041,14041,14041,14041,14041,14041,3,131
None,824258,824258,824258,824258,824258,824258,824258,824258,824258,15441,824258,824258,824258,824258,824258,824258,824258,824258,824258,824258,824258,824258,813195,824258,435603,824258,824258,824258,824256,821763,823554,824258,824258,824258,824258,824258,824258,824258,11788,23842
Order,17622,17622,17622,17622,17622,17622,17622,17622,17622,690,17622,17622,17622,17622,17622,17622,17622,17622,17622,17622,17622,17622,17432,17622,8360,17622,17622,17622,17622,17585,17618,17622,17622,17622,17622,17622,17622,17622,1209,1258
RR,9,9,9,9,9,9,9,9,9,0,9,9,9,9,9,9,9,9,9,9,9,9,9,9,7,9,9,9,9,9,9,9,9,9,9,9,9,9,0,1


In [40]:
mtd_data.groupby('mtd_predicted').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label,bundle_action_prediction,filed_prediction
mtd_predicted,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Motion,37297,37297,37297,37297,37297,37297,37297,37297,37297,1073,37297,37297,37297,37297,37297,37297,37297,37297,37297,37297,37297,37297,36784,37297,17013,37297,37297,37297,37297,37172,37277,37297,37297,37297,37297,37297,37297,37297,66,1711
None,705074,705074,705074,705074,705074,705074,705074,705074,705074,10051,705074,705074,705074,705074,705074,705074,705074,705074,705074,705074,705074,705074,695791,705074,378143,705074,705074,705074,705072,702999,704496,705074,705074,705074,705074,705074,705074,705074,4872,14835
Order,113531,113531,113531,113531,113531,113531,113531,113531,113531,5007,113531,113531,113531,113531,113531,113531,113531,113531,113531,113531,113531,113531,111928,113531,55598,113531,113531,113531,113531,113154,113419,113531,113531,113531,113531,113531,113531,113531,8062,8677
RR,28,28,28,28,28,28,28,28,28,4,28,28,28,28,28,28,28,28,28,28,28,28,28,28,23,28,28,28,28,28,28,28,28,28,28,28,28,28,0,9


# Prediction (Bundle_action)

In [88]:
db = read_data('prediction.csv')

# db = db[(db['bundle_type']=='Order')]

# db = db[(db['predicted_label']=='Order')]

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [89]:
f = db['predicted_label']=='Order'

In [90]:
d1 = db[f].copy()

In [520]:
db = db[~f]
db['bundle_action_prediction'] = db.apply(lambda x: np.nan, axis=1)


In [91]:
X1, y1 = prepro(d1,w2c,seq_len=100, emb_size=100,target_col='bundle_action')

preds,lst = prediction(model=CNN_model,weightsfile = "bundle_action.hdf5",X = X1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
# np.mean([np.argmax(x[0]) == np.argmax(x[1]) for x in zip(preds, y1)])

In [93]:
d1['bundle_action_prediction']= lst

In [94]:
values=sorted(list(set(d1['bundle_action'])))
values

['Deny', 'Grant', 'None', 'Partial']

In [95]:
# db['bundle_action_prediction']= lst2
p_class = {0:'Deny',1:'Grant',2:'None',3:'Partial'}
d1 = d1.replace({'bundle_action_prediction': p_class})

In [532]:
db = db.append(d1, ignore_index=True)

In [533]:
len(db)

855930

In [490]:
db.groupby('bundle_action').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction
bundle_action,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2246,2206,2246,1093,2246,2246,2246,2246,2243,2246,2246,2246,2246,2246,2246,2246,2246,2246
Grant,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2206,2176,2206,1069,2206,2206,2206,2206,2202,2206,2206,2206,2206,2206,2206,2206,2206,2206
None,7780,7780,7780,7780,7780,7780,707,7780,7780,7780,7780,7780,7780,7780,7780,7780,7780,7780,7780,7616,7780,3856,7780,7780,7780,7780,7754,7774,7780,7780,7780,7780,7780,7780,7780,7780
Partial,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,757,768,371,768,768,768,768,765,768,768,768,768,768,768,768,768,768


In [534]:
db.groupby('bundle_action_prediction').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted
bundle_action_prediction,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,3157,3157,3157,3157,3157,3157,2257,3157,3157,3157,3157,3157,3157,3157,3157,3157,3157,3157,3157,3098,3157,1573,3157,3157,3157,3157,3151,3155,3157,3157,3157,3157,3157,3157,3157,3157
Grant,2991,2991,2991,2991,2991,2991,2179,2991,2991,2991,2991,2991,2991,2991,2991,2991,2991,2991,2991,2945,2991,1430,2991,2991,2991,2991,2984,2991,2991,2991,2991,2991,2991,2991,2991,2991
None,5999,5999,5999,5999,5999,5999,774,5999,5999,5999,5999,5999,5999,5999,5999,5999,5999,5999,5999,5873,5999,2963,5999,5999,5999,5999,5979,5995,5999,5999,5999,5999,5999,5999,5999,5999
Partial,853,853,853,853,853,853,717,853,853,853,853,853,853,853,853,853,853,853,853,839,853,423,853,853,853,853,850,853,853,853,853,853,853,853,853,853


In [536]:
db.to_csv('prediction.csv')

# Prediction (MTD Action)

In [70]:
dd = read_data('prediction.csv')

# dd = dd[(dd['mtd_type']=='Order')]

# dd = dd[(dd['mtd_predicted']=='Order')]

In [71]:
h = dd['mtd_type']=='Order'

d3 = dd[h].copy()

dd = dd[~h]

dd['mtd_action_prediction'] = dd.apply(lambda x: np.nan, axis=1)

In [72]:
X1, y1 = prepro(d3,w2c,seq_len=100, emb_size=100,target_col='mtd_action')

preds,lst = prediction(model=CNN_model,weightsfile = "mtd_action.hdf5",X = X1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
d3['mtd_action_prediction']= lst

values=sorted(list(set(d3['mtd_action'])))
values

['Deny', 'Grant', 'Moot', 'None', 'Partial']

In [74]:
p_class = {0:'Deny',1:'Grant',2:'Moot',3:'None',4:'Partial'}
d3 = d3.replace({'mtd_action_prediction': p_class})

In [75]:
dd = dd.append(d3, ignore_index=True)

In [77]:
dd.to_csv('prediction.csv')

In [78]:
dd.groupby('mtd_action').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction,mtd_action_prediction
mtd_action,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,3982,3982,3982,3982,3982,3982,3982,3982,3982,3982,159,3982,3982,3982,3982,3982,3982,3982,3982,3982,3982,3982,3982,3937,3982,1895,3982,3982,3982,3982,3974,3982,3982,3982,3982,3982,3982,3982,3982,315,331,3959
Grant,9110,9110,9110,9110,9110,9110,9110,9110,9110,9110,260,9110,9110,9110,9110,9110,9110,9110,9110,9110,9110,9110,9110,9024,9110,4017,9110,9110,9110,9110,9092,9108,9110,9110,9110,9110,9110,9110,9110,486,505,9102
Moot,2254,2254,2254,2254,2254,2254,2254,2254,2254,2254,100,2254,2254,2254,2254,2254,2254,2254,2254,2254,2254,2254,2254,2228,2254,1134,2254,2254,2254,2254,2251,2254,2254,2254,2254,2254,2254,2254,2254,153,159,2250
None,838685,838685,838685,838685,838685,838685,838685,838685,838685,838685,15449,838685,838685,838685,838685,838685,838685,838685,838685,838685,838685,838685,838685,827475,838685,442626,838685,838685,838685,838683,836143,837977,838685,838685,838685,838685,838685,838685,838685,11805,23988,412
Partial,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,167,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,1899,1867,1899,1105,1899,1899,1899,1899,1893,1899,1899,1899,1899,1899,1899,1899,1899,241,249,1899


In [65]:
dd.groupby('mtd_action_prediction').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction
mtd_action_prediction,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Deny,4370,4370,4370,4370,4370,4370,4370,4370,4370,4370,204,4370,4370,4370,4370,4370,4370,4370,4370,4370,4370,4370,4370,4309,4370,2117,4370,4370,4370,4370,4361,4369,4370,4370,4370,4370,4370,4370,4370,4370,366,392
Grant,9466,9466,9466,9466,9466,9466,9466,9466,9466,9466,267,9466,9466,9466,9466,9466,9466,9466,9466,9466,9466,9466,9466,9390,9466,4162,9466,9466,9466,9466,9446,9463,9466,9466,9466,9466,9466,9466,9466,9466,501,517
Moot,2188,2188,2188,2188,2188,2188,2188,2188,2188,2188,94,2188,2188,2188,2188,2188,2188,2188,2188,2188,2188,2188,2188,2159,2188,1117,2188,2188,2188,2188,2186,2188,2188,2188,2188,2188,2188,2188,2188,2188,158,163
None,149,149,149,149,149,149,149,149,149,149,4,149,149,149,149,149,149,149,149,149,149,149,149,149,149,87,149,149,149,149,149,149,149,149,149,149,149,149,149,149,6,6
Partial,1449,1449,1449,1449,1449,1449,1449,1449,1449,1449,121,1449,1449,1449,1449,1449,1449,1449,1449,1449,1449,1449,1449,1425,1449,877,1449,1449,1449,1449,1443,1449,1449,1449,1449,1449,1449,1449,1449,1449,178,180


# Prediction (Filed_party)

In [560]:
dc = read_data('prediction.csv')

# dc = dc[(dc['bundle_type']!='None')]

# dc = dc[(dc['predicted_label']!='None')]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [561]:
g = dc['predicted_label']!='None'

d2 = dc[g].copy()

dc = dc[~g]

dc['filed_prediction'] = dc.apply(lambda x: np.nan, axis=1)

In [562]:
X1, y1 = prepro(d2,w2c,seq_len=100, emb_size=100,target_col='bundle_who_filed')

preds,lst = prediction(model=CNN_model,weightsfile = "bundle_filed.hdf5",X = X1)

In [563]:
d2['filed_prediction']= lst

In [564]:
values=sorted(list(set(d2['bundle_who_filed'])))
values

['None', 'X', 'defendant', 'plaintiff']

In [565]:
p_class = {0:'None',1:'X',2:'defendant',3:'plaintiff'}
d2 = d2.replace({'filed_prediction': p_class})

In [568]:
dc = dc.append(d2, ignore_index=True)

In [569]:
dc.to_csv('prediction.csv')

In [548]:
dc.groupby('bundle_who_filed').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction,filed_prediction
bundle_who_filed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
None,13783,13783,13783,13783,13783,13783,13783,5502,13783,13783,13783,13783,13783,13783,13783,13783,13783,13783,13783,13783,13516,13783,6998,13783,13783,13783,13783,13737,13777,13783,13783,13783,13783,13783,13783,13783,8914,13783
X,1595,1595,1595,1595,1595,1595,1595,1401,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1563,1595,697,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595,1514,1595
defendant,6174,6174,6174,6174,6174,6174,6174,5761,6174,6174,6174,6174,6174,6174,6174,6174,6174,6174,6174,6174,6073,6174,3050,6174,6174,6174,6174,6161,6174,6174,6174,6174,6174,6174,6174,6174,1694,6174
plaintiff,3680,3680,3680,3680,3680,3680,3680,3457,3680,3680,3680,3680,3680,3680,3680,3680,3680,3680,3680,3680,3624,3680,1737,3680,3680,3680,3680,3674,3680,3680,3680,3680,3680,3680,3680,3680,878,3680


In [549]:
dc.groupby('filed_prediction').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,activity_number,original_bundle_action,bundle_decision,bundle_motion_number,bundle_order_number,bundle_recommendation,bundle_rr_number,original_bundle_type,original_bundle_who_filed,case_number,date_filed,defendant,district,doc,docket_text,judge_id,judge_name,mag_id,mag_name,original_mtd_action,original_mtd_type,original_docket_text,plaintiff,pro_se,nature_of_suit,bundle_action,bundle_type,bundle_who_filed,mtd_action,mtd_type,clean_text,predicted_label,mtd_predicted,bundle_action_prediction
filed_prediction,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
None,8579,8579,8579,8579,8579,8579,8579,4828,8579,8579,8579,8579,8579,8579,8579,8579,8579,8579,8579,8579,8421,8579,4396,8579,8579,8579,8579,8553,8577,8579,8579,8579,8579,8579,8579,8579,8579,5515
X,2547,2547,2547,2547,2547,2547,2547,1453,2547,2547,2547,2547,2547,2547,2547,2547,2547,2547,2547,2547,2502,2547,1126,2547,2547,2547,2547,2542,2547,2547,2547,2547,2547,2547,2547,2547,2547,2322
defendant,8618,8618,8618,8618,8618,8618,8618,6123,8618,8618,8618,8618,8618,8618,8618,8618,8618,8618,8618,8618,8463,8618,4332,8618,8618,8618,8618,8594,8614,8618,8618,8618,8618,8618,8618,8618,8618,3268
plaintiff,5488,5488,5488,5488,5488,5488,5488,3717,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5488,5390,5488,2628,5488,5488,5488,5488,5478,5488,5488,5488,5488,5488,5488,5488,5488,5488,1895


In [193]:
# db.to_csv("filed_prediction.csv")

# Accuracy, Precision, and Recall

In [400]:
from sklearn.metrics import classification_report
print(classification_report(new_data["bundle_type"], new_data["predicted_label"]))


             precision    recall  f1-score   support

     Motion       0.88      1.00      0.94     10158
       None       1.00      0.99      1.00    837836
      Order       0.59      1.00      0.74      7696
         RR       0.30      0.95      0.46       240

avg / total       0.99      0.99      0.99    855930



In [401]:
from sklearn.metrics import accuracy_score
accuracy_score(new_data["bundle_type"], new_data["predicted_label"])*100

99.15670674003715

In [42]:
from sklearn.metrics import classification_report
print(classification_report(mtd_data["mtd_type"], mtd_data["mtd_predicted"]))

             precision    recall  f1-score   support

     Motion       0.37      0.99      0.54     14041
       None       1.00      0.85      0.92    824258
      Order       0.14      0.90      0.24     17622
         RR       0.29      0.89      0.43         9

avg / total       0.97      0.86      0.90    855930



In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(mtd_data["mtd_type"], mtd_data["mtd_predicted"])*100

85.64567195915554

In [96]:
from sklearn.metrics import classification_report

print(classification_report(d1["bundle_action"], d1["bundle_action_prediction"]))

             precision    recall  f1-score   support

       Deny       0.65      0.80      0.72      2246
      Grant       0.70      0.79      0.74      2206
       None       0.91      0.82      0.86      7780
    Partial       0.80      0.78      0.79       768

avg / total       0.82      0.81      0.81     13000



In [97]:
from sklearn.metrics import accuracy_score

accuracy_score(d1["bundle_action"], d1["bundle_action_prediction"])*100

80.86153846153846

In [68]:
print(classification_report(dd["mtd_action"], dd["mtd_action_prediction"]))

             precision    recall  f1-score   support

       Deny       0.68      0.75      0.71      3959
      Grant       0.85      0.88      0.86      9102
       Moot       0.60      0.58      0.59      2250
       None       0.72      0.26      0.38       412
    Partial       0.94      0.72      0.82      1899

avg / total       0.78      0.78      0.78     17622



In [69]:
accuracy_score(dd["mtd_action"], dd["mtd_action_prediction"])*100

78.12393598910454

In [566]:
print(classification_report(d2["bundle_who_filed"], d2["filed_prediction"]))

             precision    recall  f1-score   support

       None       0.94      0.58      0.72     13783
          X       0.48      0.76      0.59      1595
  defendant       0.64      0.90      0.75      6174
  plaintiff       0.57      0.85      0.69      3680

avg / total       0.78      0.71      0.71     25232



In [567]:
accuracy_score(d2["bundle_who_filed"], d2["filed_prediction"])*100

71.06848446417247